In [ ]:
import pandas as pd
import numpy as np
import traceback
import datetime
import sys

In [ ]:
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_columns', None)

In [ ]:
def import_data():
    path = "Datasets/"#diretório dos datasets
    
    # --------- as linhas abaixo servem para gerar o nome dos arquivos automaticamente
    prefix = "ABONOP_"#prefixo comun a todos os arquivos
    months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']#meses presentes
    years = ["2017", "2018", "2019", "2020"]#anos presentes
    ext = ".csv"#extensão dos arquivos
    
    # ------------------------
    
    columns = ["Nome", "CPF", "Descrição do cargo emprego", "Nível de Escolaridade", "Denominação do órgão de atuação",
              "UF da UPAG de vinculação", "Denominação unidade organizacional", "UF da Residência",
              "Cidade da residência", "Situação servidor", "Quantidade de anos no Serviço público",
              "Quantidade de meses no Serviço público","Ano/Mês inicial DO ABONO DE PERMANENCIA","VAL"] #colunas presentes
    
    date_columns = ["Ano/Mês inicial DO ABONO DE PERMANENCIA"]#colunas que devem ser transformadas em datas, porém no dataset estão como inteiros
    float_columns = {"VAL":np.float64}#coluna númerica
    
    datasets = []#esta variavel vai ser usada como uma lista de DataFrames importados
    sep = ";"#separador utilizado nos arquivos
    decimal = "."#valores decimais são separados por virgula
    try:
        name = ""
        for i in years:# esse loop vai importar os datasets ano a ano
            for j in months:# esse loop vai importar os datasets mês a mês
                name = path + prefix + j + i + ext# concatena as informações do arquivo para formar seu nome

                df = pd.read_csv(name, sep =sep, #low_memory = False,
                                names = columns, header = 0,
                                #warn_bad_lines = True
                                encoding = 'cp860', index_col=False)#ver documentação do pandas
                date = datetime.date(int(i), int(j), 1)#cria uma data do tipo 1/j/i
                df["DATE"] = [date for k in range(df.shape[0])]#adiciona uma coluna com a data
                datasets.append(df)
    except Exception as e:#ocorreu algum exceção
        traceback.print_exc()#printa a pilha de execução
        print("Erro na planilha: {}".format(name))#printa a plainlha que ocorreu o erro
    finally:
        if len(datasets) > 0:
            datasets = pd.concat(datasets)#concatena os datasets que foram importados com sucesso
            datasets.drop_duplicates(subset = columns, inplace = True)#tira as colunas duplicadas, não considera a coluna DATE
            
            datasets.reset_index(inplace = True)
            datasets.drop(columns = ["index"], inplace = True)
            
        return datasets

In [ ]:
datasets = import_data()

In [ ]:
def clean_spaces(df, text_columns, number_columns):
    for i in text_columns:
        #substitui mais de um espaço junto por apenas 1 espaço
        df[i].replace("[\s]+", " ", regex = True, inplace = True)# não retira um espaço em branco no inicio e no final se houverem
    for i in number_columns:
        #retira todos os espaços em branco
        df[i].replace("[\s]+", "", regex = True, inplace = True)
        df[i].replace("[,]", ".", regex = True, inplace = True)

In [ ]:
text_columns = ["Nome", "CPF", "Descrição do cargo emprego", "Nível de Escolaridade", "Denominação do órgão de atuação",
              "UF da UPAG de vinculação", "Denominação unidade organizacional", "UF da Residência",
              "Cidade da residência", "Situação servidor"]
number_columns = ["Quantidade de anos no Serviço público","Quantidade de meses no Serviço público",
                  "Ano/Mês inicial DO ABONO DE PERMANENCIA","VAL"]

In [ ]:
clean_spaces(datasets, text_columns, number_columns)

In [ ]:
datasets